Packages importeren

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import pyodbc
import sqlite3

Connectie maken met Data Warehouse

In [ ]:
DB = {'servername': 'ENVY-AKRAM\\SQLEXPRESS','database': 'Great Outdoors - Akram en Sylvan'}
export_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')
export_cursor = export_conn.cursor()
export_cursor

Connectie maken met brongegevens

In [ ]:
go_sales_connection = sqlite3.connect('Data\\Brongegevens\\go_sales.sqlite')
go_crm_connection = sqlite3.connect('Data\\Brongegevens\\go_crm.sqlite')
go_staff_connection = sqlite3.connect('Data\\Brongegevens\\go_staff.sqlite')
df_inventory = pd.read_csv('Data\\Brongegevens\\GO_SALES_INVENTORY_LEVELSData.csv')
df_product = pd.read_csv('Data\\Brongegevens\\GO_SALES_PRODUCT_FORECASTData.csv')

Helper functie tables inlezen

In [ ]:
def readTable(connection, table):
    query = 'SELECT * FROM ' + table
    result = pd.read_sql_query(query, connection)
    result = result.loc[:, ~result.columns.str.contains('trial', case=False)]
    #verwijder trials
    return result

Database Go Sales

In [ ]:
tables = pd.read_sql('SELECT name FROM sqlite_master WHERE type="table"', go_sales_connection)
country_go_sales = readTable(go_sales_connection, 'country')
order_details = readTable(go_sales_connection, 'order_details')
order_header = readTable(go_sales_connection, 'order_header')
order_method = readTable(go_sales_connection, 'order_method')
product = readTable(go_sales_connection, 'product')
product_line = readTable(go_sales_connection, 'product_line')
product_type = readTable(go_sales_connection, 'product_type')
retailer_site_go_sales = readTable(go_sales_connection, 'retailer_site')
return_reason = readTable(go_sales_connection, 'return_reason')
sales_branch_go_sales = readTable(go_sales_connection, 'sales_branch')
sales_staff_go_sales = readTable(go_sales_connection, 'sales_staff')
sales_targetdata = readTable(go_sales_connection, 'SALES_TARGETData')
tables  


Database Go Staff

In [ ]:
tables = pd.read_sql('SELECT name FROM sqlite_master WHERE type="table"', go_staff_connection)
course = readTable(go_staff_connection, 'course')
sales_branch_go_staff = readTable(go_staff_connection, 'sales_branch')
sales_staff_go_staff = readTable(go_staff_connection, 'sales_staff')
satisfaction = readTable(go_staff_connection, 'satisfaction')
satisfaction_type = readTable(go_staff_connection, 'satisfaction_type')
training = readTable(go_staff_connection, 'training')
tables


Database Go Crm

In [ ]:
tables = pd.read_sql('SELECT name FROM sqlite_master WHERE type="table"', go_crm_connection)
age_group = readTable(go_crm_connection, 'age_group')
country_go_crm = readTable(go_crm_connection, 'country')
retailer = readTable(go_crm_connection, 'retailer')
retailer_contact = readTable(go_crm_connection, 'retailer_contact')
retailer_headquarters = readTable(go_crm_connection, 'retailer_headquarters')
retailer_segment = readTable(go_crm_connection, 'retailer_segment')
retailer_site_go_crm = readTable(go_crm_connection, 'retailer_site')
retailer_type = readTable(go_crm_connection, 'retailer_type')
sales_demographic = readTable(go_crm_connection, 'sales_demographic')
sales_territory = readTable(go_crm_connection, 'sales_territory')
tables

retailer_headquarters


# Unions

In [ ]:
sales_staff_union = pd.concat([sales_staff_go_staff, sales_staff_go_sales])
sales_staff_union.drop_duplicates(subset='SALES_STAFF_CODE', ignore_index=True, inplace=True)
sales_staff_union
#heeft overwegingen nodig op: 
#manager_code


In [ ]:
sales_branch_union = pd.concat([sales_branch_go_staff, sales_branch_go_sales])
sales_branch_union.drop_duplicates(subset='SALES_BRANCH_CODE', ignore_index=True, inplace=True)
sales_branch_union

In [ ]:
retailer_site_union = pd.concat([retailer_site_go_crm, retailer_site_go_sales])
retailer_site_union.drop_duplicates(subset='RETAILER_SITE_CODE', ignore_index=True, inplace=True)
retailer_site_union

In [ ]:
country_go_crm_union = pd.merge(country_go_crm, sales_territory, on='SALES_TERRITORY_CODE', how='outer')
country_union = pd.merge(country_go_sales, country_go_crm_union, on='COUNTRY_CODE', how='outer')
country_union.drop_duplicates(subset='COUNTRY_CODE', ignore_index=True, inplace=True)
country_union
#not actually a union

# Verborgen Dimensies

Date

In [ ]:
date_dict = {'DATE_DATE':[], 'DATE_YEAR':[], 'DATE_QUARTER':[], 'DATE_MONTH':[], 'DATE_DAY':[]}

date_dw = pd.DataFrame(date_dict)
date_dw

ToDate() Functions

In [ ]:
#??

Month

In [ ]:
#iets

# Simpele Dimensies

Return Reason

In [ ]:
def ToReasonCategory(str):

    if 'wrong' in str.lower():
        return 'Wrong Product'
    elif 'unsatisfactory' in str.lower():
        return 'Unsatisfactory'
    else:
        return 'Faulty'

return_reason_dw = return_reason.copy()
return_reason_dw['RETURN_CATEGORY'] = return_reason_dw['RETURN_DESCRIPTION_EN'].apply(ToReasonCategory)
return_reason_dw

Course

In [ ]:
def ToCourseCategory(str):

    if 'marketing' in str.lower():
        return 'Marketing'
    elif 'sales' in str.lower():
        return 'Sales'
    elif 'management' in str.lower():
        return 'Management'
    elif 'communication' in str.lower():
        return 'Communication'
    elif 'orientation' in str.lower():
        return 'Orientation'
    else: 
        return 'Other'

course_dw = course.copy()
course_dw['COURSE_CATEGORY'] = course_dw['COURSE_DESCRIPTION'].apply(ToCourseCategory)
course_dw


Order Method

In [ ]:
order_method_dw = order_method.copy()
#fuck de categorie

Satisfaction Type

In [ ]:
def ToSatisfied(nr):
    try:
        return int(nr) >= 3
    except ValueError:
        return False

satisfaction_type_dw = satisfaction_type.copy
satisfaction_type_dw['SATISFIED_BOOLEAN'] = satisfaction_type['SATISFACTION_TYPE_CODE'].apply(ToSatisfied)
satisfaction_type_dw

# Merges (dimensies)

Retailer Headquarters

In [ ]:
retailer_headquarters_dw = pd.merge(
    retailer_headquarters.add_prefix('RT_HEADQUARTERS_'), country_union.add_prefix('COUNTRY_'), 
    left_on = 'RT_HEADQUARTERS_COUNTRY_CODE', right_on='COUNTRY_COUNTRY_CODE', how='outer')
retailer_headquarters_dw = pd.merge(
    retailer_headquarters_dw, retailer_segment.add_prefix('RT_SEGMENT_'), 
    left_on = 'RT_HEADQUARTERS_SEGMENT_CODE', right_on='RT_SEGMENT_SEGMENT_CODE', how='outer')
#segment category
retailer_headquarters_dw

Retailer Contact

In [ ]:
retailer_contact_dw = pd.merge(
    retailer_contact.add_prefix('RT_CONTACT_'), retailer_site_union.add_prefix('RT_SITE_'), 
    left_on='RT_CONTACT_RETAILER_SITE_CODE', right_on='RT_SITE_RETAILER_SITE_CODE', how='inner')
#position category
retailer_contact_dw = pd.merge(
    retailer_contact_dw, retailer.add_prefix('RT_'), 
    left_on='RT_SITE_RETAILER_CODE', right_on='RT_RETAILER_CODE', how='inner')
retailer_contact_dw = pd.merge(
    retailer_contact_dw, retailer_type.add_prefix('RT_TYPE_'), 
    left_on='RT_RETAILER_TYPE_CODE', right_on='RT_TYPE_RETAILER_TYPE_CODE', how='inner')
retailer_contact_dw = pd.merge(
    retailer_contact_dw, retailer_headquarters_dw, 
    left_on='RT_RETAILER_CODEMR', right_on='RT_HEADQUARTERS_RETAILER_CODEMR', how='inner')
retailer_contact_dw
#trial222/888
#loc: any non americas?

Sales Staff

In [ ]:
sales_staff_dw = pd.merge(
    sales_staff_union.add_prefix('SALES_STAFF_'), sales_branch_union.add_prefix('SALES_BRANCH_'), 
    left_on='SALES_STAFF_SALES_BRANCH_CODE', right_on='SALES_BRANCH_SALES_BRANCH_CODE', how='inner')
sales_staff_dw = pd.merge(
    sales_staff_dw, country_go_sales.add_prefix('COUNTRY_'),
    left_on='SALES_BRANCH_COUNTRY_CODE', right_on='COUNTRY_COUNTRY_CODE', how='inner')
sales_staff_dw
#trial633/888

Retailer Headquarters

In [ ]:
#wat doen we hiermee?
"""
retailer_headquarters = pd.merge(
    retailer_headquarters[['RETAILER_CODEMR', 'RETAILER_NAME', 'ADDRESS1', 'ADDRESS2', 'POSTAL_ZONE', 'CITY', 'COUNTRY_CODE', 'REGION', 'PHONE', 'FAX', 'SEGMENT_CODE']], 
    country[['COUNTRY_CODE', 'COUNTRY_EN', 'FLAG_IMAGE', 'SALES_TERRITORY_CODE']], left_on='COUNTRY_CODE', right_on='COUNTRY_CODE', 
    how='inner'
    ).merge(sales_territory[['SALES_TERRITORY_CODE', 'TERRITORY_NAME_EN']], left_on='SALES_TERRITORY_CODE', right_on='SALES_TERRITORY_CODE', how='inner'
            ).merge(retailer_segment[['SEGMENT_CODE', 'LANGUAGE', 'SEGMENT_NAME', 'SEGMENT_DESCRIPTION']], left_on='SEGMENT_CODE', right_on='SEGMENT_CODE', how='inner')
retailer_headquarters
"""

Product

In [ ]:
#zijn de koloomen nodig?
#prefixes toevoegen
product_dw = pd.merge(
    product[['PRODUCT_NUMBER', 'PRODUCT_NAME', 'PRODUCT_IMAGE', 'DESCRIPTION', 'INTRODUCTION_DATE', 'PRODUCTION_COST', 'MARGIN', 'LANGUAGE', 'PRODUCT_TYPE_CODE']], 
    product_type[['PRODUCT_TYPE_CODE', 'PRODUCT_TYPE_EN', 'PRODUCT_LINE_CODE']], 
    left_on='PRODUCT_TYPE_CODE', right_on='PRODUCT_TYPE_CODE', how='inner'
    ).merge( 
        product_line[['PRODUCT_LINE_CODE', 'PRODUCT_LINE_EN']], 
        left_on='PRODUCT_LINE_CODE', right_on='PRODUCT_LINE_CODE', how='inner')
product_dw

Sales Demographic

In [ ]:
#prefixes
sales_demographic = pd.merge(
    sales_demographic[['DEMOGRAPHIC_CODE', 'SALES_PERCENT', 'RETAILER_CODEMR', 'AGE_GROUP_CODE']]
    , age_group[['AGE_GROUP_CODE', 'UPPER_AGE', 'LOWER_AGE']], left_on='AGE_GROUP_CODE', right_on='AGE_GROUP_CODE', how='inner')
sales_demographic

# Merges (feiten)

Sales Target

In [ ]:
sales_targetdata_dw = pd.DataFrame()

sales_targetdata_dw['SALES_TARGETDATA_CODE'] = sales_targetdata['Id']
sales_targetdata_dw['SALES_TARGETDATA_SALES_TARGET'] = sales_targetdata['SALES_TARGET']
sales_targetdata_dw['MONTH_YEAR'] = sales_targetdata['SALES_PERIOD']
sales_targetdata_dw['MONTH_MONTH'] = sales_targetdata['SALES_YEAR']
sales_targetdata_dw['PRODUCT_PRODUCT_NUMBER'] = sales_targetdata['PRODUCT_NUMBER']
sales_targetdata_dw['SALES_STAFF_SALES_STAFF_CODE'] = sales_targetdata['SALES_STAFF_CODE']
sales_targetdata_dw['RETAILER_RETAILER_CODE'] = sales_targetdata['RETAILER_CODE']

sales_targetdata_dw
#take month from MONTH
#ETL klopt niet - sales branch en order method verbinden niet

Inventory Levels

In [ ]:
inventory_levels_dw = pd.DataFrame()



Training

In [ ]:
#prefixes
training = pd.merge(
    course[['COURSE_CODE', 'COURSE_DESCRIPTION']],
    training[['COURSE_CODE', 'SALES_STAFF_CODE', 'YEAR']], left_on='COURSE_CODE', right_on='COURSE_CODE', how='inner')
training

Satisfaction

In [ ]:
#prefixes
satisfaction = pd.merge(
    satisfaction[['YEAR', 'SALES_STAFF_CODE', 'SATISFACTION_TYPE_CODE']],
    satisfaction_type[['SATISFACTION_TYPE_CODE', 'SATISFACTION_TYPE_DESCRIPTION']], left_on='SATISFACTION_TYPE_CODE', right_on='SATISFACTION_TYPE_CODE', how='inner')
satisfaction

Order

In [ ]:
#->>

Returned Item

In [ ]:
#->>

Product Forecast

In [ ]:
#->>